In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from collections import Counter

##############################
# Definition of Env variable # 
##############################
#Definition of the current env
# JUPYTER or G_COLAB
CURRENT_ENV = "G_COLAB"

#Use on Jupyter_env only 
DATA_DIR = "../../dev_GenJavaBdd/GenerationTest/"

# Lecture des fichiers
get the number of sbt/bdd function

In [ ]:
if CURRENT_ENV == "JUPYTER":
    list_folder = os.listdir(DATA_DIR)
    num_folder = len(list_folder)-2
elif CURRENT_ENV == "G_COLAB":
    with open("bdd.txt","r") as f:
        num_folder = len(f.readlines())
    with open("sbt.txt","r") as f:
        if (len(f.readlines()) != num_folder):
            error("[ERROR] number of BDD function does not match BDD number")
print("Number of function/Foler = ", num_folder)  # This would give length of files.


# Creation des listes 

In [ ]:
from tqdm import tqdm
from time import sleep

In [ ]:
bdd = []
sbt = []

#Create the list and save it as file
if CURRENT_ENV == "JUPYTER":
    print("Get the bdd and sbt list via folder")
    
    # Sleep to prevent fail tqdm print
    #sleep(1)
    #for i  in tqdm (range(num_folder), desc="loading..."):
    for i  in range(num_folder):
        path_iteration = DATA_DIR+'/Dir_Fonction_'+str(i+1)
        bdd_path = path_iteration+"/BDD_Fonction_"+str(i+1)+".txt"
        stb_path = path_iteration+"/SBT_Generation_Output/Fonction_"+str(i+1)+"_SBT.txt"
        bdd_txt = open(bdd_path,'r').read()
        sbt_txt = open(stb_path,'r').read()
        clean_bdd = cleaning_bdd(bdd_txt)
        bdd.append(clean_bdd)
        sbt.append(sbt_txt)
    #sleep(1)
    
    print("Save the BDD & SBT list as file")
    # Save the list as file
    #os.remove("bddi.txt")
    f_bdd = open("bdd.txt", "w+")
    f_bdd.write('\n'.join(bdd))
    f_bdd.close()

    #os.remove("sbt.txt")
    f_sbt = open("sbt.txt", "w+")
    f_sbt.write('\n'.join(sbt))
    f_sbt.close()
    
#Create the list from the txt file
else:
    print("Get the bdd and sbt list via the bdd.txt & sbt.txt")
    with open("bdd.txt") as f:
        tmp_bdd = f.read()
    bdd = tmp_bdd.split("\n")

    with open("sbt.txt") as f:
        tmp_sbt = f.read()
    sbt = tmp_sbt.split("\n")


In [ ]:
# Cette fonction sera utilisée pour récupérer les codes sbt et bdd dans un autre code python
def return_bdd_sbt():
    return bdd,sbt

In [ ]:
bdd2,_ = return_bdd_sbt()

# Recuperation des valeurs max de bdd et sbt

In [ ]:
def return_max_len( text ):
    all_len = []
    for unique_code in text:
        code_split = unique_code.split(' ')
        all_len.append(len(code_split))
    return max(all_len)

In [ ]:
bdd_counter =Counter([word for sentence in bdd for word in sentence.split(' ')])
sbt_counter =Counter([word for sentence in sbt for word in sentence.split(' ')])

In [ ]:
def return_common_words(counter):
    common_words = []
    for word in counter:
        occurence = counter[word]
        if occurence > 3500:
            common_words.append(word)
    return common_words

In [ ]:
common_words_bdd = return_common_words(bdd_counter)
common_words_sbt = return_common_words(sbt_counter)

common_words_bdd.append('<oov>')
common_words_sbt.append('<oov>')
common_words_bdd.append('sostoken ')
common_words_bdd.append(' eostoken')
common_words_bdd.append(' ')
common_words_sbt.append(' ')

#print('Common words SBT : \n',common_words_sbt)
#print('\n')
#print('Common words BDD : \n',common_words_bdd)

In [ ]:
word_to_ind_sbt = {word : ind for ind,word in enumerate(common_words_sbt)}
ind_to_word_sbt = {ind : word for ind,word in enumerate(common_words_sbt)}

word_to_ind_bdd = {word : ind for ind,word in enumerate(common_words_bdd)}
ind_to_word_bdd = {ind : word for ind,word in enumerate(common_words_bdd)}

In [ ]:
number_folder = len(bdd)
max_words_sbt = return_max_len(sbt)+1
max_words_bdd = return_max_len(bdd)+1
number_of_words_sbt = len(common_words_sbt)
number_of_words_bdd = len(common_words_bdd)
'''
print(' number of forlder :',number_folder)
print(' number maximum of word in one sbt code :',max_words_sbt)
print(' number maximum of word in one bdd code :',max_words_bdd)
print(' number of different word in sbt :',number_of_words_sbt)
print(' number of different word in bdd :',number_of_words_bdd)
'''

# ---------------------------------------------------------------------------------------------------------------

# Encodage BDD

In [ ]:
def return_listing(text,common_words): 
    arr= [[] for _ in range(num_folder)] #pour calculer la taille max de toute les variables réunies 
    var = [] # pour creer un vocabulaire 
    for i,sentence in  enumerate(text):
        text_split = sentence.split( ' ')
        for j,word in enumerate (text_split):
            if word not in common_words:
                arr[i].append(word)
                var.append(word) 
    return arr, var

In [ ]:
arr_var_bdd, variables_bdd = return_listing(bdd,common_words_bdd)

In [ ]:
arr_var_sbt,variables_sbt= return_listing(sbt,common_words_sbt)

Conversion des variables en sentences, puis on transforme ce tablea de sentences en dataframe pour pouvoir appliquer les sostoken et eosteoken

In [ ]:
def return_txt_trous(text,common_words):
    
    txt_trous = []
    
    
    for i,words in enumerate(text):
        temp_sentence = []
        split_words = words.split(' ')
        for j,word in enumerate(split_words):
            if word not in common_words:
                temp_sentence.append('<oov>')
                
            if word in common_words:
                temp_sentence.append(word)
                            
        txt_trous.append(' '.join(temp_sentence))
    return txt_trous

In [ ]:
def return_txt_trous_unique(text):
    
    txt_trous = []  
    split_words = text.split(' ')
    for j,word in enumerate(split_words):
        if word not in common_words_sbt:
            txt_trous.append('<oov>')

        if word in common_words_sbt:
            txt_trous.append(word)

    return ' '.join(txt_trous)

In [ ]:
txt_trou_unique = return_txt_trous_unique(sbt[0])

In [ ]:
txt_trou_bdd = return_txt_trous(bdd, common_words_bdd)
txt_trou_sbt = return_txt_trous(sbt, common_words_sbt)

In [ ]:
df = pd.DataFrame(txt_trou_bdd, columns=['bdd'])
df['sbt'] = txt_trou_sbt
df['bdd_input'] = txt_trou_bdd
df['bdd_label'] = txt_trou_bdd
df.tail(5)

# Calcul du nombre de var max par code et du nombre de lettre total

In [ ]:
def return_nb_var_maxlen_var(arr):
    all_len = []
    nb_var = []
    for words in arr:
        temp_len = 0
        for j,word in enumerate(words):
            for i,char in enumerate(word):
                _
            temp_len=temp_len+i+1
        nb_var.append(j+1)
        all_len.append(temp_len)
    return max(nb_var), max(all_len)

# Creation d'un vocabulaire pour les variables 

In [ ]:

encoder_input = np.array(df.sbt)
decoder_input = np.array(df.bdd_input)
decoder_label = np.array(df.bdd_label)


indices = np.arange(num_folder)
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_label = decoder_label[indices]

df.head(10)

In [ ]:
total = num_folder
test_size = 0.1

train_encoder_input = encoder_input[:-int(total*test_size)]
train_decoder_input = decoder_input[:-int(total*test_size)]
train_decoder_label = decoder_label[:-int(total*test_size)]

test_encoder_input = encoder_input[-int(total*test_size):]
test_decoder_input = decoder_input[-int(total*test_size):]
test_decoder_label = decoder_label[-int(total*test_size):]
'''
print("train dataset shape")
print(train_encoder_input.shape)
print(train_decoder_input.shape)
print(train_decoder_label.shape)

print("\n\ntest dataset shape")
print(test_encoder_input.shape)
print(test_decoder_input.shape)
print(test_decoder_label.shape)
'''

Il faut maintenant convertir ce code en sequences, puis padded ces sequences, on va pas ajouter les tokens de debut et fin maintenant, on les ajoutera apres avoir encodé

In [ ]:
def encode(text,word_to_ind, dim1, dim2,type):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix = np.full((dim1,dim2),word_to_ind[' '])
    #print('Encoding Matrix shape : ',encoded_matrix.shape)
    for i,words in enumerate(text):
        j=0
        #print(vars)
        split_word = words.split(' ')
        if type == 1:
            encoded_matrix[i,j]=word_to_ind['sostoken ']
            j=j+1
        
        for word in (split_word):
            encoded_matrix[i,j]=word_to_ind[word]

            j=j+1
        if type == 2:
            encoded_matrix[i,j]=word_to_ind[' eostoken']
            j=j+1

    return encoded_matrix

In [ ]:
def encode_unique(text):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix_unique = np.full(256,word_to_ind_sbt[' '])

    split_word = text.split(' ')

    for i,word in enumerate(split_word):
        encoded_matrix_unique[i]=word_to_ind_sbt[word]

    return encoded_matrix_unique

In [ ]:
txt_trou_unique

In [ ]:
encode_unique(txt_trou_unique)

In [ ]:
#print("train dataset shape")
train_encoder_input_encoded = encode(train_encoder_input, word_to_ind_sbt,len(train_encoder_input),max_words_sbt,0)
train_decoder_input_encoded = encode(train_decoder_input, word_to_ind_bdd,len(train_decoder_input),max_words_bdd,1)
train_decoder_label_encoded = encode(train_decoder_label, word_to_ind_bdd,len(train_decoder_label),max_words_bdd,2)
#print("\n\ntest dataset shape")

test_encoder_input_encoded = encode(test_encoder_input, word_to_ind_sbt,len(test_encoder_input),max_words_sbt,0)
test_decoder_input_encoded = encode(test_decoder_input, word_to_ind_bdd,len(test_decoder_input),max_words_bdd,1)
test_decoder_label_encoded = encode(test_decoder_label, word_to_ind_bdd,len(test_decoder_label),max_words_bdd,2)

In [ ]:
train_encoder_input_encoded.shape

In [ ]:
def decode ( encoded_sequence, ind_to_char):
    decoded_sequence = []
    for var in encoded_sequence:
        decoded_sequence.append(ind_to_char[var])
    return ' '.join(decoded_sequence)

In [ ]:
decode(train_decoder_label_encoded[0],ind_to_word_bdd)

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense,Concatenate,Attention
from keras.models import Model
#from keras.utils import plot_model
from keras import backend as K

#hyperparameters
embedding_size = 256
hidden_size = 256

# trainer model (generator model will use the same encoder tho)
encoder_input = Input(shape=[max_words_sbt]) # size chain SBT 
encoder_embedding = Embedding(max_words_sbt,embedding_size,mask_zero=True)
encoder_embedded = encoder_embedding(encoder_input)

encoder_lstm1 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output1,encoder_h1,encoder_c1 = encoder_lstm1(encoder_embedded)

encoder_lstm2 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output2,encoder_h2,encoder_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
encoder_output3,encoder_h3,encoder_c3 = encoder_lstm3(encoder_output1)

decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(max_words_bdd,embedding_size,mask_zero=True) # len chain bdd
decoder_embedded = decoder_embedding(decoder_input)

decoder_lstm = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0.2)
decoder_output,_,_ = decoder_lstm(decoder_embedded,initial_state=[encoder_h3,encoder_c3])

attn_layer = Attention()
attn_context = attn_layer([decoder_output,encoder_output3])

decoder_output = Concatenate(axis=-1)([decoder_output,attn_context])
tanh_dense= Dense(hidden_size*2,activation=K.tanh)
decoder_output = tanh_dense(decoder_output)

softmax_dense = Dense(number_of_words_bdd,activation='softmax')
decoder_output = softmax_dense(decoder_output)

trainer_model = Model([encoder_input,decoder_input],decoder_output)
trainer_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#trainer_model.summary()

In [ ]:
#trainer_hist =trainer_model.fit([train_encoder_input_encoded,train_decoder_input_encoded],train_decoder_label_encoded,epochs=5,batch_size=128,validation_split=0.2)

In [ ]:
#trainer_model.save_weights('template_attention.h5')
trainer_model.load_weights('template_attention.h5')

In [ ]:
#generator model
gen_encoder = Model([encoder_input],[encoder_output3,encoder_h3,encoder_c3])

gen_decoder_values_input = Input(shape=(max_words_sbt,hidden_size))
gen_decoder_h_input = Input(shape=[hidden_size])
gen_decoder_c_input = Input(shape=[hidden_size])

gen_decoder_embedded = decoder_embedding(decoder_input)
gen_decoder_output,gen_decoder_h,gen_decoder_c = decoder_lstm(gen_decoder_embedded,initial_state=[gen_decoder_h_input,gen_decoder_c_input])

attn_context = attn_layer([gen_decoder_output,gen_decoder_values_input])
gen_decoder_output = Concatenate(axis=-1)([gen_decoder_output,attn_context])

gen_decoder_output = tanh_dense(gen_decoder_output)
gen_decoder_output = softmax_dense(gen_decoder_output)

gen_decoder = Model([decoder_input]+[gen_decoder_values_input,gen_decoder_h_input,gen_decoder_c_input],[gen_decoder_output,gen_decoder_h,gen_decoder_c])

# Test

In [ ]:
def generate_from_encoder_input(encoder_input):
    encoder_input = encoder_input.reshape(1,-1)
    values,h,c = gen_encoder.predict(encoder_input)
    
    single_tok = np.zeros((1,1))
    single_tok[0,0] = word_to_ind_bdd['sostoken ']
    decoder_input = single_tok
    
    generated = []
    count = 0
    while(True):
        decoder_output,new_h,new_c = gen_decoder.predict([decoder_input]+[values,h,c])
        count +=1
        
        sampled_index = np.argmax(decoder_output[0,-1,:])
        sampled_word = ind_to_word_bdd[sampled_index]
        
        if sampled_word != ' eostoken' and sampled_index != 0:
            generated.append(sampled_word)
        if count >= max_words_bdd or sampled_word == ' eostoken':
            break
        
        h,c = new_h,new_c
        decoder_input[0,0] = sampled_index
    
    generated = ' '.join(generated)
    return generated

In [ ]:
'''
for i in range(520,525):
    print("\n<<sample encoder input SBT >>")
    print(decode(train_encoder_input_encoded[i],ind_to_word_sbt))
    print("\n")
    print("<<generated BDD >>")
    print(generate_from_encoder_input(train_encoder_input_encoded[i]))
    print("\n")
    print("<<Expected BDD>>")
    print(decode(train_decoder_label_encoded[i],ind_to_word_bdd))
    print("========================================\n")
'''

In [ ]:
input_encoded = test_encoder_input_encoded[0]

In [ ]:
def return_generated_template(input_encoded):
    generated = generate_from_encoder_input(input_encoded)
    generated = generated.split(' ')
    return generated

In [ ]:
score = trainer_model.evaluate([train_encoder_input_encoded,train_decoder_input_encoded], train_decoder_label_encoded, verbose=0)
print("%s: %.2f%%" % (trainer_model.metrics_names[1], score[1]*100))